In [24]:
import pandas as pd
import streamlit as st
import joblib
import base64

# Função para codificar a imagem em base64
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()

# Caminho da imagem de fundo
background_image_path = 'C:/Users/User/OneDrive - MSFT/Documents/Python Impressionador/Modulo 41 - Ciência de Dados/Aula 2 - projeto e recomendações/background.jpg'
background_image_base64 = get_base64_of_bin_file(background_image_path)

# Adicionando a imagem de fundo e estilizando os campos com CSS
page_bg = f'''
<style>
.stApp {{
background-image: url("data:image/jpg;base64,{background_image_base64}");
background-size: cover;
background-position: center;
}}

input, select {{
    background-color: rgba(255, 255, 255, 0); /* Totalmente transparente */
    color: white; /* Texto dentro das caixas */
    border: 1px solid white; /* Borda branca */
}}

textarea {{
    background-color: rgba(255, 255, 255, 0); 
    color: white;
    border: 1px solid white;
}}

input[type=number] {{
    -webkit-appearance: none;
    -moz-appearance: textfield;
}}
</style>
'''

st.markdown(page_bg, unsafe_allow_html=True)

# Definição das variáveis de entrada
x_numericos = {
    'latitude': 0,
    'longitude': 0,
    'accommodates': 0,
    'bathrooms': 0,
    'bedrooms': 0,
    'beds': 0,
    'extra_people': 0,
    'minimum_nights': 0,
    'ano': 0,
    'mes': 0,
    'n_amenities': 0,
    'host_listings_count': 0
}

x_tf = {'host_is_superhost': 0, 'instant_bookable': 0}

x_listas = {
    'property_type': ['Apartment', 'Bed and breakfast', 'Condominium', 'Guest suite', 'Guesthouse', 'Hostel', 'House', 'Loft', 'Outros', 'Serviced apartment'],
    'room_type': ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
    'cancellation_policy': ['flexible', 'moderate', 'strict', 'strict_14_with_grace_period']
}

dicionario = {}

# Receber entradas dos campos numéricos com títulos em negrito
for i, item in enumerate(x_numericos):
    st.markdown(f"<label style='color:white;'><strong>{item}</strong></label>", unsafe_allow_html=True)
    if item == 'latitude' or item == 'longitude':
        valor = st.number_input('', step=0.000001, value=0.0, format="%.5f", key=f'num_input_{i}')
    elif item == 'extra_people':
        valor = st.number_input('', step=0.01, value=0.0, key=f'num_input_{i}')
    else:
        valor = st.number_input('', step=1, value=0, key=f'num_input_{i}')                       
    x_numericos[item] = valor

# Receber entradas dos campos booleanos com títulos em negrito
for item in x_tf:
    st.markdown(f"<label style='color:white;'><strong>{item}</strong></label>", unsafe_allow_html=True)
    valor = st.selectbox('', ('Sim', 'Não'), key=item)
    x_tf[item] = 1 if valor == 'Sim' else 0

# Receber entradas das listas de opções com títulos em negrito
for item in x_listas:
    st.markdown(f"<label style='color:white;'><strong>{item}</strong></label>", unsafe_allow_html=True)
    valor = st.selectbox('', x_listas[item], key=item)
    dicionario[f'{item}_{valor}'] = 1
    
# Botão de sugestão
botao = st.button("Sugestão do valor de diária")    

if botao:
    dicionario.update(x_numericos)
    dicionario.update(x_tf)
    valores_x=pd.DataFrame(dicionario, index=[0])
    
    # Ler o CSV
    dados = pd.read_csv("dados.csv")
    # Excluir a primeira e última colunas (unnamed: 0 e price)
    colunas = list(dados.columns)[1:-1]
    
    # Garantir que as colunas que existem em 'dados' também estejam presentes em 'valores_x'
    for coluna in colunas:
        if coluna not in valores_x.columns:
            valores_x[coluna] = 0  # Adiciona a coluna com valor 0 se não existir

    # Agora reordenar as colunas
    valores_x = valores_x[colunas]
    
    # Carregar e usar o modelo preditivo
    modelo = joblib.load('modelo.joblib')
    preco = modelo.predict(valores_x)
    
    st.markdown(f"<h2 style='color:#FE504F;'>Sugestão de valor da diária: R$ {preco[0]:.2f}</h2>", unsafe_allow_html=True)






2024-10-09 22:55:29.552 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-10-09 22:55:29.554 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-10-09 22:55:29.560 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-10-09 22:55:29.561 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-10-09 22:55:29.561 `label` got an empty value. 

   host_is_superhost  host_listings_count  latitude  longitude  accommodates  \
0                  0                    0         0          0             0   

   bathrooms  bedrooms  beds  extra_people  minimum_nights  ...  \
0          0         0     0             0               0  ...   

   room_type_Entire home/apt  room_type_Hotel room  room_type_Private room  \
0                          1                     0                       0   

   room_type_Shared room  bed_type_Outros  bed_type_Real Bed  \
0                      0                0                  0   

   cancellation_policy_flexible  cancellation_policy_moderate  \
0                             1                             0   

   cancellation_policy_strict  cancellation_policy_strict_14_with_grace_period  
0                           0                                                0  

[1 rows x 34 columns]
